In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import cv2
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from tkinter import Tk, Label, Button, filedialog
from PIL import Image, ImageTk


In [ ]:
model = YOLO('yolov8n.pt')  # You can choose small, medium, or large model

results = model('test_image.jpg')
boxes = results[0].boxes

def get_dominant_color(image, k=2):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = image.reshape((-1, 3))
    clt = KMeans(n_clusters = k)
    clt.fit(image)
    return clt.cluster_centers_[0]  # Dominant color
    
def classify_color(rgb_color):
    r, g, b = rgb_color
    if b > 100 and b > r and b > g:
        return 'blue'
    return 'other'
    
if predicted_color == 'blue':
    rectangle_color = (0, 0, 255)  # Red in BGR
else:
    rectangle_color = (255, 0, 0)  # Blue in BGR


In [ ]:
def open_image():
    path = filedialog.askopenfilename()
    image = cv2.imread(path)
    process_and_display(image)

def annotate_frame(image, results):
    for box in results[0].boxes:
        cls_id = int(box.cls[0])
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        if cls_id == 2:  # Car
            car_crop = image[y1:y2, x1:x2]
            color = classify_color(get_dominant_color(car_crop))
            if color == 'blue':
                color_box = (0, 0, 255)  # Red box for blue car
            else:
                color_box = (255, 0, 0)  # Blue box for other
            cv2.rectangle(image, (x1, y1), (x2, y2), color_box, 2)
        elif cls_id == 0:  # Person
            cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
    return image
    

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models

# Data generators
train_dir = '/kaggle/input/car_colors/train'
val_dir = '/kaggle/input/car_colors/val'

train_datagen = ImageDataGenerator(rescale=1./255, zoom_range=0.2, horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir, target_size=(64, 64), batch_size=32, class_mode='categorical')
val_generator = val_datagen.flow_from_directory(val_dir, target_size=(64, 64), batch_size=32, class_mode='categorical')

# Model architecture
model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(64, 64, 3)),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(len(train_generator.class_indices), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

history = model.fit(train_generator, validation_data=val_generator, epochs=10)
model.save('car_color_classifier.h5')

from tensorflow.keras.preprocessing import image
import numpy as np

def predict_car_color(img_path, model, class_indices):
    img = image.load_img(img_path, target_size=(64, 64))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    
    pred = model.predict(img_array)[0]
    class_labels = list(class_indices.keys())
    return class_labels[np.argmax(pred)]
    
